## Библиотека Typing

Питон всё ещё считается языком без строгой типизации. Это значит, что для того, чтобы завести переменную не надо заранее указывать ее тип, а сам тип хранимого значения может поменяться по ходу работы программы.

In [1]:
import re

import pymorphy2
from pymorphy2.analyzer import MorphAnalyzer

In [2]:
a = 1
print(a, type(a))
a = 1.
print(a, type(a))
a = '1'
print(a, type(a))


1 <class 'int'>
1.0 <class 'float'>
1 <class 'str'>


Это серьезно упрощает разработку, не накладывая строгих ограничений на очень многие процессы. С другой стороны, программисту гораздо проще сделать ошибку. В примере ниже мы забыли, что `а` приняло значение строки. Язык со строгой типизацией отказался бы выполнять операцию, так как типы сравниваемых значений не совпадают, а Питон ничего, выполнил. Теперь проблема на нашей стороне.

С другой стороны, подобное поведение избавляет нас от массы проблем, например, переписывать функцию несколько раз для различных типов данных.

In [3]:
if a == 1:
    print('Correct')
else:
    print('It wents wrong.')

It wents wrong.


Однако контроль типов - весьма полезная операция, отданная на откуп сторонним программам. Позже мы с вами познакомимся с линтерами, которые будут указывать на подобные ошибки. 

Мы уже увидели, что писать типы переменных - это хороший тон. Но пока мы моем писать только конкретный тип. Что делать, если конкретный тип неизвестен? Например, если в функцию может передаваться любой итерируемый объект?

Для этого была разработана библиотека `typing`, которая хранит в себе объявления разных типов. Документация на библиотеку находится [здесь](https://docs.python.org/3/library/typing.html), дополнительные материалы в двух частях [здесь](https://habr.com/ru/company/lamoda/blog/432656/) и [здесь](https://habr.com/ru/company/lamoda/blog/435988/).

Если какая-либо функция может не возвращать значение, или само значение может не передаваться в функцию, используется модификатор `Optional`. Теперь соответствующая функция, которая перебирает кодировки и возвращает `None`, если кодировка не была найдена, будет выглядеть вот так. 

In [4]:
from typing import Optional

def checkAllEncodings(self) -> Optional[str]: # Теперь функция возвращает Optional[str], то есть str или None.
    for encoding in self.allowedEncodings:
        if checkEncoding(encoding):
            return encoding
    return None


Если переменная, параметр или возвращаемое значение могут быть любого типа, следует использовать `Any`.

In [5]:
from typing import Any

def print_status(status: Any) -> None:
    print(f'Current status is: {str(status)}')
    
print_status(1)
print_status('good')

Current status is: 1
Current status is: good


Если мы можем работать с ограниченным списком типов, используется модификатор `Union`.

In [6]:
from typing import Union

def add_numbers(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    return a + b

print(add_numbers(1, 1)) # int
print(add_numbers(1., 1)) # float

2
2.0


Для списков, кортежей и словарей можно использовать стандартные типы, но можно и типы из `typing`: `List, Tuple, Dict`.

In [7]:
from typing import List, Tuple, Dict

def summ_items(data: Union[List, Tuple, Dict]) -> Union[int, float]:
    return sum(data)

print(summ_items([1, 2, 3, 4]))
print(summ_items((1, 2, 3, 4)))
print(summ_items({10:1, 20:2, 30:3, 40:4}))

10
10
100


Вообще, я хотел сказать, что в эту функцию можно передать любой итерируемый объект. Используем для этого `Iterator`.

In [8]:
# from collections.abc import Iterator
from typing import Iterator

def summ_items(data: Iterator) -> Iterator:
    return sum(data)

print(summ_items([1, 2, 3, 4]))
print(summ_items((1, 2, 3, 4)))
print(summ_items({10:1, 20:2, 30:3, 40:4}))

10
10
100


Если мы хотим показать, что в качестве типа используется функция или любой вызываемый объект, можно использовать `Callable`, у которого в квадратных скобках будет список передаваемых параметров и возвращаемое значение: `Callable[[ArgType1, ArgType2,...], ReturnType]`. Кстати, в соотетствующем классе я не прописал список свойств именно из-за того, что непонятно было, какой тип писать свойству-ссылке на функцию.

In [9]:
from typing import Callable

# Ключевое слово class после которого идет название нашего класса.
class FasterMorphologyUnified:
    
    morpho: MorphAnalyzer
    cash: dict
    analyzeWords: Callable[[FasterMorphologyUnified, list], list]
        
    def __init__(self, dict_type: str ="PyM") -> None: # Функция инициализации объекта после его создания.
        # Создаем новую морфологию в каждом объекте. 
        # А вдруг мы будем потом работать с разными языками? У каждого объекта должна быть своя.
        if dict_type == 'PyM':
            self.morpho = pymorphy2.MorphAnalyzer() 
            self.cash = {} # Создаем словарь для кеширования.
            self.analyzeWords = self.analyzeWordsWithPymorphy
        elif dict_type == 'MyS':
            self.mystem = pymystem3.Mystem()
            self.analyzeWords = self.analyzeWordsWithMystem
        # Вообще-то надо предусмотреть вариант, если нам передали какое-то еще значение, которого мы не знаем.
        self.mode = dict_type # Сохраним, чтобы потом можно было понять что за словарь использовался.
            

NameError: name 'FasterMorphologyUnified' is not defined

И вот тут возникает проблема. Мы не можем использовать имя класса внутри объявления этого класса, так как объявление ещё не закончено и новый тип не создан (а ну как всё закончится ошибкой?). В таком случае тип надо записать в виде строки, Питон тогда отложит поиск типа на потом.



In [10]:
from typing import Callable

# Ключевое слово class после которого идет название нашего класса.
class FasterMorphologyUnified:
    
    morpho: MorphAnalyzer
    cash: dict
    analyzeWords: Callable[['FasterMorphologyUnified', list], list]
        
    def __init__(self, dict_type: str ="PyM") -> None: # Функция инициализации объекта после его создания.
        # Создаем новую морфологию в каждом объекте. 
        # А вдруг мы будем потом работать с разными языками? У каждого объекта должна быть своя.
        if dict_type == 'PyM':
            self.morpho = pymorphy2.MorphAnalyzer() 
            self.cash = {} # Создаем словарь для кеширования.
            self.analyzeWords = self.analyzeWordsWithPymorphy
        elif dict_type == 'MyS':
            self.mystem = pymystem3.Mystem()
            self.analyzeWords = self.analyzeWordsWithMystem
        # Вообще-то надо предусмотреть вариант, если нам передали какое-то еще значение, которого мы не знаем.
        self.mode = dict_type # Сохраним, чтобы потом можно было понять что за словарь использовался.
            

<h2>Векторизация текстов</h2>

Теперь напишем класс, отвечающий за векторизацию текста.

Для определения меры сходства двух текстов используется косинусная мера сходства, рассчитываемая по следующей формуле: $cos(a,b)=\frac{\sum{a_i * b_i}}{\sqrt {\sum{a_i^2}*\sum{b_i^2}}}$.<br>
Вообще-то, использовать стандартную функцию рассчета косинусной меры сходства из <a href="http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html">sklearn</a> было бы быстрее. Но мне хотелось показать как работать с разными типами входа.

In [11]:
import math
import numpy as np

In [12]:
# Другое имя класса, так как он обладает несколько иной функциональностью.
class FasterMorphology2:
    """ Класс для быстрого морфологического анализа текстов и их векторизации. 
    """
    
    def __init__(self) -> None: # Функция инициализации объекта после его создания.
        self.morpho = pymorphy2.MorphAnalyzer()
        self.__cash = {}
        self.__dictionary = {} # Добавим словарь для запоминания, на каком месте вектора находится какая начальная форма.
        
    def analyzeWords(self, words: list) -> list:
        """ Проводит морфологический анализ списка токенов words.
            Возвращает список начальных форм слов.
        """
        res: list
            
        res = []
        for w in words:
            if w in self.__cash: # Сперва ищем очередное слово в кеше.
                res.append(self.__cash[w])
            else: # Если его там нет, проводим морфологический анализ и кешируем.
                r = self.morpho.parse(w)[0].normal_form
                res.append(r)
                self.__cash[w] = r
                if r not in self.__dictionary: # Также для каждой начальной формы запоминаем ее позицию в векторе.
                    self.dictionary[r] = len(self.dictionary) + 1
        return res
    
    def analyzeText(self, text: str) -> list:
        """ Проводит морфологический анализ строки с текстом text. 
            Выделяет из нее слова, написанные русской кириллицей.
            Возвращает список начальных форм слов.
        """
        words: list
        
        words = [w[0] for w in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
        return self.analyzeWords(words)
        
    # Вообще-то тоже самое умеет Counter, но ему надо сперва привести слова к начальной форме.
    def vectorizeAsDict(self, words) -> dict:
        """ Возвращает векторное разреженное представление текста в виде словаря.
            Текст передается как список токенов words.
            Вместо позиции для индексации используется само слово.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        vct: dict
        res: list
        
        vct = {}
        res = []
        for w in words: # Для каждого слова проводим анализ.
            if w in self.__cash:
                vct[self.__cash[w]] = vct.get(self.__cash[w], 0) + 1 # Считаем частоты слов.
            else:
                r = self.morpho.parse(w)[0].normal_form
                res.append(r)
                self.__cash[w] = r
                vct[r] = vct.get(r, 0) + 1
                if r not in self.__dictionary:
                    self.__dictionary[r] = len(self.__dictionary)
        return vct
    
    def clearDict(self):
        """ Очищает словарь. Вдруг надо пересчитать так как изменилась размерность пространства.
        """
        self.dictionary = {}
    
    def formDict(self, texts: list):
        """ Сформировать словарь по тексту не формируя разметку текста.
        """
        for text in texts:
            for word in text:
                if word not in self.cash:
                    r = self.morpho.parse(w)[0].normal_form
                    self.__cash[word] = r
                    if r not in self.__dictionary:
                        self.__dictionary[r] = len(self.__dictionary)
    
    def vectorizeAsList(self, words: list) -> list:
        """ Возвращает векторное представление текста в виде плотного списка (включает нули).
            Текст передается как список токенов words.
            Позиция каждого слова в векторе определяется числом, хранимым в dictionary.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        # Сперва обновляем dictionary.
        for word in words:
            if word not in self.__cash:
                r = self.morpho.parse(w)[0].normal_form
                self.__cash[word] = r
                if r not in self.__dictionary:
                    self.__dictionary[r]=len(self.__dictionary.keys())
        # Теперь, когда все слова есть в кеше и словаре и известен размер вектора, можно приступать к векторизации.
        vct = [0 for _ in self.__dictionary]
        for word in words:
            vct[self.__dictionary[self.__cash[word]]] += 1
        return vct
    
    def vectorizeAsList2(self, words: list) -> list:
        """ Возвращает векторное представление текста в виде плотного списка (включает нули).
            Текст передается как список токенов words. В вектор включаются только слова, находящиес в словаре.
            Позиция каждого слова в векторе определяется числом, хранимым в dictionary.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        vct = [0 for _ in self.__dictionary]
        for word in words:
            if word in self.__cash:
                vct[self.__dictionary[self.__cash[word]]] += 1
        return vct

    def vectorizeAsArray(self, words: list) ->list:
        """ Возвращает векторное представление текста в виде плотного массива (включает нули).
            Текст передается как список токенов words.
            Позиция каждого слова в векторе определяется числом, хранимым в dictionary.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        # Сперва обновляем dictionary.
        for word in words:
            if word not in self.__cash:
                r = self.morpho.parse(w)[0].normal_form
                self.cash[word] = r
                if r not in self.__dictionary:
                    self.__dictionary[r]=len(self.__dictionary.keys())
        # Теперь, когда все слова есть в кеше и словаре и известен размер вектора, можно приступать к векторизации.
        vct = np.zeros((len(self.__dictionary)))
        for word in words:
            vct[self.__dictionary[self.__cash[word]]] += 1
        return vct

    # Здесь мы заложили проблему. Функция не умеет считать расстояние между np.array.
    # А ещё эти две функции не имеют никакого отношения к морфологическому анализу.
    # Но представим себе, что мы завели ещё две функции, которые считают расстояние между текстами, а не векторами.
    def cosineSimilarity(self, a, b) -> float:
        """ Функция расчета косинусной меры сходства между двумя векторными представлениями текста.
            Работает по-разному в зависимости от представления вектора.
        """
        if type(a) != type(b): # Тип векторов должен совпадать.
            return None
        if isinstance(a, list): # Если это списки, значит это плотное представление вектора.
            if 0 == len(a) or 0 == len(b) or len(a) != len(b): # Длины векторов в этом случае должны совпадать.
                return 0
            sumab = sum([a[na] * b[na] for na in range(len(a))])
            suma2 = sum([a[na] * a[na] for na in range(len(a))])
            sumb2 = sum([b[na] * b[na] for na in range(len(a))])
            return sumab / math.sqrt(suma2 * sumb2)        
        elif isinstance(a, dict): # Разреженное представление вектора - хранятся только ненулевые значения.
            if 0 == len(a.keys()) or 0 == len(b.keys()): # Вектора должны хранить хоть что-то.
                return 0
            sumab = sum([a[na] * b[na] for na in set(a.keys()) & set(b.keys())])
#            sumab=sum([a[na]*b[na] for na in a.keys() if na in b.keys()])
            suma2 = sum([a[na] * a[na] for na in a.keys()])
            sumb2 = sum([b[nb] * b[nb] for nb in b.keys()])
            return sumab / math.sqrt(suma2 * sumb2)  
        return 0
    
    def JaccardCoefficient(self, a, b) -> float:
        """ Коэффициент Жаккара - отношение количества слов, встречающихся в обоих текстах к объединению лексики.
        """
        if type(a) != type(b): # Тип векторов должен совпадать.
            return None
        if isinstance(a, list): # Если это списки, значит это плотное представление вектора.
            if 0 == len(a) or 0 == len(b) or len(a) != len(b): # Длины векторов в этом случае должны совпадать.
                return 0
            union = len(a) - [aa * bb for aa, bb in zip(a, b)].count(0)
            intersection = len(a) - [aa + bb for aa, bb in zip(a, b)].count(0)
            return union / intersection
        elif isinstance(a, dict): # Разреженное представление вектора - хранятся только ненулевые значения.
            if 0 == len(a.keys()) or 0 == len(b.keys()): # Вектора должны хранить хоть что-то.
                return 0
            return len(set(a.keys()) & set(b.keys())) / len(set(a.keys()) | set(b.keys()))
        return 0
        

#### Замечения к классу

Вообще-то, он страшный. Там смешан словарь и векторизация текстов. Вообще-то, это должно быть два разных класса.

С другой стороны, если от всей этой ваашей морфологии нам нужны только векторы, то вроде бы сойдет, даже быстрее должно работать. Беда в том, что есть решения и побыстрее. Так что это просто учебный класс, который показывает как оно может векторизоваться. И чтобы показать, что можно использовать разреженное представление векторов.

Посмотрим как работает новая морфология. Возьмем также один из "Севастопольских рассказов" того же автора, чтобы было что использовать при расчете косинусной меры сходства.

In [13]:
faster3 = FasterMorphology2()

In [16]:
with open('data/war_and_peace.txt') as fil:
    textWP = fil.read()
# Выделяем все слова написанные русской кириллицей.
words = [w[0] for w in re.findall('([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)', textWP)]
newtext = ' '.join(words)

with open('data/sebastopol.txt') as fil:
    textSb = fil.read()
words3 = [w[0].lower() for w in re.findall('([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)', textSb)]
newtext3 = ' '.join(words3)

Создадим плотное и разреженное представление для двух текстов.

In [17]:
vd1 = faster3.vectorizeAsDict(words)
vd2 = faster3.vectorizeAsDict(words3)
vl1 = faster3.vectorizeAsList(words)
vl2 = faster3.vectorizeAsList(words3)

Посмотрим как быстро считается разреженное и плотное предствления.

In [18]:
%time
r1 = faster3.cosineSimilarity(vd1, vd2)
%time
r2 = faster3.cosineSimilarity(vl1, vl2)
print(r1, r2)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.58 µs
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs
0.9561367711772639 0.9561367711772639


In [19]:
print(vl1.count(0), vl2.count(0), len(vl1))

529 15350 18840


In [20]:
%time
r1 = faster3.JaccardCoefficient(vd1, vd2)
%time
r2 = faster3.JaccardCoefficient(vl1, vl2)
print(r1, r2)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs
0.1571656050955414 0.1571656050955414


Посмотрим сколько наши вектора занимают памяти.

In [21]:
import sys

print(sys.getsizeof(vd1))
print(sys.getsizeof(vl1))

589920
153752


Разница по памяти в 4 раза. Попробуем с numpy.array.

In [22]:
va1 = faster3.vectorizeAsArray(words)
print(sys.getsizeof(va1))

150832


Здесь какой-то небольшой выигрыш. Мораль - проще разработать свой собственный класс для разреженного хранения на основе общего для всех словаря с индексом и двумя массивами для индекса слов конкретного текста и их частот.

Хорошо, продолжим с косинусной мерой. Попробуем посчитать сходство с "Марсианином" Энди Вейра.

In [23]:
with open('data/veyr/index_split_017.xhtml') as fil: # Грузим главу 17, она побольше.
    textM17 = fil.read()
words4 = [w[0].lower() for w in re.findall('([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)', textM17)]
newtext4 = ' '.join(words4)

In [24]:
# Считаем вектора.
vd3 = faster3.vectorizeAsDict(words4)
vl3 = faster3.vectorizeAsList(words4)

In [25]:
%%time
print(faster3.cosineSimilarity(vd1, vd3))
print(faster3.cosineSimilarity(vd2, vd3))

0.7853540382103844
0.7776684489226509
CPU times: user 5.73 ms, sys: 213 µs, total: 5.95 ms
Wall time: 5.72 ms


In [26]:
%%time
print(faster3.cosineSimilarity(vl1, vl3))
print(faster3.cosineSimilarity(vl2, vl3))

0
0
CPU times: user 41 µs, sys: 5 µs, total: 46 µs
Wall time: 45.1 µs


Что-то пошло не так. Постараемся понять что именно.

In [27]:
sumab = sum([vl1[na] * vl3[na] for na in range(len(vl3))])
suma2 = sum([vl1[na] * vl1[na] for na in range(len(vl3))])
sumb2 = sum([vl3[na] * vl3[na] for na in range(len(vl3))])
sumab / math.sqrt(suma2 * sumb2) 

IndexError: list index out of range

Ну конечно же! Надо же пересчитать все вектора, а то размерность пространства изменилась! Со словарями такой проблемы не было.

In [28]:
vl1 = faster3.vectorizeAsList(words)
vl2 = faster3.vectorizeAsList(words3)

In [29]:
%%time
print(faster3.cosineSimilarity(vl1, vl3))
print(faster3.cosineSimilarity(vl2, vl3))

0.7853540382103844
0.7776684489226509
CPU times: user 7.38 ms, sys: 0 ns, total: 7.38 ms
Wall time: 7.3 ms


In [30]:
vl1.count(0), vl2.count(0), vl3.count(0), len(vl1)

(956, 15777, 17677, 19267)

Теперь попробуем построить вектора для всего текста "Марсианина".

In [31]:
words5 = []
for i in range(2, 33):
    with open(f'data/veyr/index_split_0{i:0>2}.xhtml') as fil:
        textMar = fil.read()
    words6 = [w[0].lower() for w in re.findall('([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)', textMar)]
    words5 += words6
newtext5 = ' '.join(words5)

In [32]:
vd4 = faster3.vectorizeAsDict(words5)
vl4 = faster3.vectorizeAsList(words5)

In [33]:
vl1 = faster3.vectorizeAsList(words)
vl2 = faster3.vectorizeAsList(words3)
vl3 = faster3.vectorizeAsList(words4)

In [34]:
%%time
print(faster3.cosineSimilarity(vd1, vd4))
print(faster3.cosineSimilarity(vd2, vd4))
print(faster3.cosineSimilarity(vd3, vd4))

0.8039365043485874
0.8083884676412758
0.8816806000490218
CPU times: user 8.92 ms, sys: 100 µs, total: 9.02 ms
Wall time: 9 ms


Да, глава из "Марсианина" больше похожа на всё произведение, чем на Толстого. Но общая мера сходства довольно большая. То есть по-хорошему, речь идет в большой степени об одинаковых вещах.

In [35]:
%%time
print(faster3.cosineSimilarity(vl1, vl4))
print(faster3.cosineSimilarity(vl2, vl4))
print(faster3.cosineSimilarity(vl3, vl4))

0.8039365043485874
0.8083884676412758
0.8816806000490218
CPU times: user 11.4 ms, sys: 0 ns, total: 11.4 ms
Wall time: 11.2 ms


In [36]:
print(len(vl1) - vl1.count(0), 
      len(vl1) - vl2.count(0), 
      len(vl1) - vl3.count(0), 
      len(vl1) - vl4.count(0), 
      len(vl1))
print(len(words), len(words5))

18311 3490 1590 8011 22241
445508 90841


In [37]:
with open('data/war_and_peace3.txt') as fil:
    textWP = fil.read()
words6 = [w[0] for w in re.findall('([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)', textWP)]
print(len(words6))
faster4 = FasterMorphology2()
vd5 = faster4.vectorizeAsDict(words6)
vl5 = faster4.vectorizeAsList(words6)
print(len(vl5))

104178
8736


### Ещё два слова про производительность

Как уже было сказано выше, вычисление косинуса при помощи SciPy будет быстрее. На самом деле почти всё будет быстрее, чем тот код, что написан выше, но мне хотелось сравнимых решений, демонстрирующих внутреннюю структуру метода.

А теперь давайте посмотрим как можно сделать быстрее.

In [38]:
# Создадим случайный массив
arr_size = 1000
a = np.random.rand(100)
ad = {i:a[i] for i in range(100)}


In [39]:
%%timeit
# Вот так считается часть косинусной меры у нас.
s = sum([ad[i]*ad[i] for i in ad.keys()])

10.1 µs ± 157 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [40]:
%%timeit
# Используем скалярное произведение векторов для того, чтобы посчитать сумму поэлементных произведений.
s = np.dot(np.array(list(ad.values())), np.array(list(ad.values())))

8.2 µs ± 201 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [41]:
%%timeit
# Этот небольшой трюк сократит время расчетов в два раза.
a2 = np.array(list(ad.values()))
s = np.dot(a2, a2)

4.48 µs ± 109 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [42]:
%%timeit
s = np.dot(a, a) # А если бы мы с самого начала использовали numpy...

753 ns ± 10.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


<h2>CountVectorizer и TfidfVectorizer</h2>

На самом деле примерно всё то же самое можно селать при помощи класса CountVectorizer из sklearn.feature_extraction.text. При помощи функции <i>fit\_transform</i> можно получить разреженное представление матрицы частот слов. Основная проблема состоит в том, что индексы в матрице представляют собой индексы в словаре переданных текстов. Сам словарь хранится в свойстве <i>vocabulary\_</i> и умеет возвращать индекс по слову (но не наоборот).

In [45]:
from sklearn.feature_extraction.text import CountVectorizer

In [46]:
counter = CountVectorizer()
# Просим посчитать частоты слов.
res = counter.fit_transform([newtext, newtext3, newtext5])
# Разреженное представление счетчика.
print(res[0][0,:10])
# Можно получить индекс по слову, ...
print(counter.vocabulary_.get('левый'))
# ... но не наоборот.
print(counter.vocabulary_.get(20342))

  (0, 5)	10
  (0, 6)	7
  (0, 7)	3
  (0, 0)	1
  (0, 2)	1
  (0, 4)	1
  (0, 3)	1
  (0, 9)	3
20342
None


Более того, CountVectorizer просто выделяет подстроки и ничего не знает про морфологию (ее можно правильно прикрутить, но это хлопотное занятие). Зато он умеет выделять n-граммы (n слов идущих подряд (или даже букв)). Помимо этого, можно попросить выдать все подстроки, создав анализатор. И можно сказать как выделять подстроки при помощи регулярного выражения.

In [47]:
def getMeaningfullWords(text: str, morph: MorphAnalyzer):
    words = []
    tokens = re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', text)
    for t in tokens:
        pv = morph.parse(t)
        if pv[0].tag.POS in ['ADJF', 'NOUN', 'VERB', 'PRTF', 'GRND']:
            words.append(pv[0].normal_form)
    return words

lemmaCounter = CountVectorizer(ngram_range=(1,3), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')

morph = MorphAnalyzer()

c = [' '.join(getMeaningfullWords(newtext, morph)),
     ' '.join(getMeaningfullWords(newtext3, morph)),
     ' '.join(getMeaningfullWords(newtext5, morph))]
analyze = lemmaCounter.build_analyzer()
res1 = analyze(c[0])
res2 = lemmaCounter.fit_transform(c)

In [48]:
print(res1[:10])

['лев', 'николаевич', 'толстой', 'война', 'мир', 'тот', 'часть', 'первый', 'быть', 'поместье']


Теперь попробуем другой показатель для подсчета важности слов в тексте - $TF*IDF$. Здесь $TF$ - Term Frequency, частота термина в документе, а $IDF$ - Inverted Document Frequency, обратная частота термина в коллекции (количество документов, в которых встречается данный термин).

Идея метрики очень проста. Если слово встречается почти во всех документах - его различительная сила очень мала и само слово не является важным. Если слово часто встречается в данном документе, то оно являетсяя важным для него.

Метрика считается на коллекции документов для каждого слова, каждого документа. Для расчета меры можно использовать `TfidfVectorizer`, который работает так же как `CountVectorizer`.

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [50]:
lemmaCounter = TfidfVectorizer(ngram_range=(1,3), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')

c = [' '.join(getMeaningfullWords(newtext, morph)),
     ' '.join(getMeaningfullWords(newtext3, morph)),
     ' '.join(getMeaningfullWords(newtext5, morph))]
analyze = lemmaCounter.build_analyzer()
res1 = analyze(c[0])
res2 = lemmaCounter.fit_transform(c)

In [51]:
print(res2[0][0,:10])

  (0, 2)	0.00011569805879228521
  (0, 5)	0.00011569805879228521
  (0, 1)	0.00011569805879228521
  (0, 4)	0.00011569805879228521
  (0, 9)	0.00023139611758457042
  (0, 0)	0.00011569805879228521
  (0, 3)	0.00011569805879228521


Попробуем посмотреть на близость глав "Марсианина" при помощи косинусной меры по частотам слов.

In [52]:
wordsM = []
for i in range(2, 33):
    with open(f'data/veyr/index_split_0{i:0>2}.xhtml') as fil:
        textWP = fil.read()
    words6 = [w[0].lower() for w in re.findall('([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)', textWP)]
    wordsM.append(words6)


In [53]:
m_vects = []

for words in wordsM:
    m_vects.append(faster3.vectorizeAsDict(words))


In [54]:
nearest = -1
txt1 = -1
txt2 = -1
for i, vct1 in enumerate(m_vects):
    for j, vct2 in enumerate(m_vects):
        if vct1 is vct2:
            continue
        cc = faster3.cosineSimilarity(vct1, vct2)
        if cc > nearest:
            nearest = cc
            txt1 = i
            txt2 = j
print(nearest, txt1+2, txt2+2)

0.9420221325678382 22 25


А теперь возьмем косинусную меру сходства по результатам TF*IDF.

In [55]:
textM = []
for i in range(2, 33):
    with open(f'data/veyr/index_split_0{i:0>2}.xhtml') as fil:
        textM.append(fil.read())

In [56]:
lemmaCounter = CountVectorizer(ngram_range=(1,3), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')
#lemmaCounter=TfidfVectorizer(ngram_range=(1,3), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')

analyze = lemmaCounter.build_analyzer()
#res1=analyze(c[0])
res2 = lemmaCounter.fit_transform(textM)

In [57]:
from sklearn.metrics.pairwise import cosine_similarity

In [58]:
%%time
nearest = -1
txt1 = -1
txt2 = -1
for i, vct1 in enumerate(res2):
    for j, vct2 in enumerate(res2):
        if i==j:
            continue
        cc = cosine_similarity(vct1, vct2)
        if cc > nearest:
            nearest = cc
            txt1 = i
            txt2 = j
print(nearest, txt1+2, txt2+2)



[[0.85149324]] 15 21
CPU times: user 1.03 s, sys: 5 µs, total: 1.03 s
Wall time: 1.03 s


In [59]:
%%time
cc2 = cosine_similarity(res2, res2)
np.fill_diagonal(cc2, 0.)
pos = np.argmax(cc2)
print(cc2[pos//res2.shape[0], pos%res2.shape[0]], pos//res2.shape[0]+2, pos%res2.shape[0]+2)

0.851493236611201 15 21
CPU times: user 21.8 ms, sys: 19 µs, total: 21.8 ms
Wall time: 20.8 ms
